# Sprint1 課題 機械学習フロー

In [29]:
"""
Home Credit Default Riskの分類問題
標準化まで。データは5万件使用
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df_train = pd.read_csv('../application_train.csv').sample(frac=1)[:50000]
df_X = df_train.drop("TARGET", axis=1)
df_Y = df_train[["TARGET"]]

#カテゴリーを数値変換
categorical_feats = [
    f for f in df_X.columns if df_X[f].dtype == 'object'
]
categorical_feats_ = categorical_feats.copy()

for f in categorical_feats_:
    df_X[f], _ = pd.factorize(df_X[f])
    df_X[f] = df_X[f].astype('int')
    
#残りの欠損値を平均で埋める
df_X = df_X.fillna(df_X.mean())

#データ分割
(X_train, X_test, 
     y_train, y_test) = train_test_split(df_X.values, df_Y.values.flatten(), test_size=0.3)

#標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## 【問題1】クロスバリデーション

In [30]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
kf.get_n_splits(X_train)

5

In [31]:
for a, b in kf.split(X_train):
    print('分割データA:{}\n分割データB{}'.format(a, b))

分割データA:[ 7000  7001  7002 ... 34997 34998 34999]
分割データB[   0    1    2 ... 6997 6998 6999]
分割データA:[    0     1     2 ... 34997 34998 34999]
分割データB[ 7000  7001  7002 ... 13997 13998 13999]
分割データA:[    0     1     2 ... 34997 34998 34999]
分割データB[14000 14001 14002 ... 20997 20998 20999]
分割データA:[    0     1     2 ... 34997 34998 34999]
分割データB[21000 21001 21002 ... 27997 27998 27999]
分割データA:[    0     1     2 ... 27997 27998 27999]
分割データB[28000 28001 28002 ... 34997 34998 34999]


## 【問題2】グリッドサーチ

### GridSearchによるパラメータ探索および学習

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

"""GridSearchCVの使い方
GridSearchCV(estimator,
             param_grid,
             scoring=None,
             fit_params=None,
             n_jobs=1,
             iid=True,
             refit=True,
             cv=None,
             verbose=0,
             pre_dispatch=‘2*n_jobs’,
             error_score=’raise’,
             return_train_score=’warn’
            )
estimator: チューニングを行うモデル
param_grid: パラメータ候補値を「パラメータ名, 候補値リスト」の辞書で与える
n_jobs: 同時実行数(-1にするとコア数で同時実行)
refit: Trueだと最良だったパラメータを使い学習データ全体で再学習する
cv: Cross validationの分割数(default: 3分割)
verbose: ログ出力レベル
"""


#LogisticRegressionモデルのパラメータを羅列
diparameter = {
    "C": [10**i for i in range(-2,4)],
    "random_state": [123],
    "class_weight": ['balanced', None],
}

grid_search = GridSearchCV(LogisticRegression(),
                           param_grid=diparameter,
                           cv=5,
                           scoring='roc_auc',
                          )
grid_search.fit(X_train, y_train)

/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3

/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.01, 0.1, 1, 10, 100, 1000], 'random_state': [123], 'class_weight': ['balanced', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [33]:
#最良の点数
grid_search.best_score_

0.7388844955761332

In [34]:
#　最良のパラメータの組み合わせ
grid_search.best_params_

{'C': 0.01, 'class_weight': None, 'random_state': 123}

In [35]:
grid_search.best_estimator_

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=123, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### GridSearchCVで最良とでたパラメータで予測

In [36]:
#最良のパラメータ
# clf_logistic_gridsearched = LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
#           intercept_scaling=1, max_iter=100, multi_class='warn',
#           n_jobs=None, penalty='l2', random_state=123, solver='warn',
#           tol=0.0001, verbose=0, warm_start=False).fit(X_train, y_train)

y_predict_logistic_gridsearched = grid_search.best_estimator_.predict(X_test)
y_predict_proba_logistic_gridsearched = grid_search.best_estimator_.predict_proba(X_test)

In [37]:
estimation_logistic_dict = {
    "accuracy": accuracy_score(y_test, y_predict_logistic_gridsearched),
    "precision": precision_score(y_test, y_predict_logistic_gridsearched),
    "recall": recall_score(y_test, y_predict_logistic_gridsearched),
    "f1":f1_score(y_test, y_predict_logistic_gridsearched)
}

estimation_logistic = pd.DataFrame(estimation_logistic_dict, index=['LogisticRegression'])
estimation_logistic

,accuracy,precision,recall,f1
LogisticRegression,0.9184,0.411765,0.005733,0.011309


In [39]:
"""
テストデータを読み込み、処理
"""

df_apptest = pd.read_csv('../application_test.csv')

#カテゴリーを数値変換
categorical_feats = [
    f for f in df_apptest.columns if df_apptest[f].dtype == 'object'
]
categorical_feats_ = categorical_feats.copy()

for f in categorical_feats_:
    df_apptest[f], _ = pd.factorize(df_apptest[f])
    df_apptest[f] = df_apptest[f].astype('int')
    
#残りの欠損値を平均で埋める
df_apptest = df_apptest.fillna(df_apptest.mean())

X_apptest = df_apptest.values

#標準化
scaler.fit(X_apptest)
X_apptest = scaler.transform(X_apptest)

In [40]:
#予測
y_apt_predict_proba_logistic_gridsearched = grid_search.best_estimator_.predict_proba(X_apptest)

In [41]:
"""
csvファイルへ書き込み
"""

import csv as csv

submit_file = open("home_credit_submit_190202_2.csv", "w")
file_object = csv.writer(submit_file)
file_object.writerow(["SK_ID_CURR", "TARGET"])
file_object.writerows(zip(df_apptest["SK_ID_CURR"].values, y_apt_predict_proba_logistic_gridsearched[:, 1]))
submit_file.close()

#### kaggle　AUCは0.73450

In [42]:
from sklearn.model_selection  import RandomizedSearchCV
from scipy.stats import uniform

param_dist = {
    "C": uniform(loc=0, scale=4),
    "penalty": ['l2'],
    "class_weight": ['balanced', None],
}

random_search = RandomizedSearchCV(
    LogisticRegression(),
   param_distributions=param_dist,
   cv=5,
   scoring='roc_auc',
)
random_search.fit(X_train, y_train)

/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3

/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a35c907b8>, 'penalty': ['l2'], 'class_weight': ['balanced', None]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [43]:
random_search.best_estimator_

LogisticRegression(C=0.028018634499291206, class_weight='balanced',
          dual=False, fit_intercept=True, intercept_scaling=1,
          max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
          random_state=None, solver='warn', tol=0.0001, verbose=0,
          warm_start=False)

In [44]:
random_search.best_score_

0.7375063706146369

In [45]:
#テストデータで予測
y_predict_proba_logistic_random = random_search.best_estimator_.predict_proba(X_test)
y_predict_proba_logistic_random

array([[0.5871687 , 0.4128313 ],
       [0.73007828, 0.26992172],
       [0.52633612, 0.47366388],
       ...,
       [0.69158633, 0.30841367],
       [0.32884981, 0.67115019],
       [0.68211309, 0.31788691]])

In [46]:
#kaggleのテストデータを予測
y_apt_predict_proba_logistic_random = random_search.best_estimator_.predict_proba(X_apptest)

In [47]:
"""
csvファイルへ書き込み
"""

submit_file = open("home_credit_submit_190202_3.csv", "w")
file_object = csv.writer(submit_file)
file_object.writerow(["SK_ID_CURR", "TARGET"])
file_object.writerows(zip(df_apptest["SK_ID_CURR"].values, y_apt_predict_proba_logistic_random[:, 1]))
submit_file.close()

kaggleでAUCは0.73448

## 【問題3】Kernelからの調査

kernelから特徴量のデータにマイナス、異常値を変更するというものがあったため試すことにする。

## 【問題4】高い汎化性能のモデル

In [48]:
pd.set_option("display.max_columns", 120)
df_train.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,50000.000000,50000.000000,50000.000000,5.000000e+04,5.000000e+04,49999.000000,4.996600e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,16925.000000,50000.0,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,21885.000000,49893.000000,40076.000000,24673.000000,20799.000000,25679.000000,16814.000000,15079.000000,23427.000000,24892.000000,25172.000000,16172.000000,20383.000000,15902.000000,24948.000000,15368.000000,22449.000000,24673.000000,20799.000000,25679.000000,16814.000000,15079.000000,23427.000000,24892.000000,25172.000000,16172.000000,20383.000000,15902.000000,24948.000000,15368.000000,22449.000000,24673.000000,20799.000000,25679.000000,16814.000000,15079.000000,23427.000000,24892.000000,25172.000000,16172.000000,20383.000000,15902.000000,24948.000000,15368.000000,22449.000000,25937.000000,49847.000000,49847.000000,49847.000000,49847.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,43198.000000,43198.000000,43198.000000,43198.000000,43198.000000,43198.000000
mean,278884.000200,0.079820,0.417880,1.702558e+05,6.000058e+05,27194.199044,5.391498e+05,0.020789,-16036.542840,63363.810600,-4978.119460,-2991.549680,12.099025,1.0,0.821120,0.20214,0.998260,0.282240,0.059440,2.153140,2.055040,2.034740,12.048740,0.01448,0.049300,0.040160,0.076640,0.228760,0.178140,0.500997,0.514076,0.510049,0.117054,0.088272,0.978064,0.751369,0.043817,0.077927,0.149791,0.225735,0.230515,0.065809,0.099530,0.106807,0.008455,0.028745,0.113815,0.087358,0.977465,0.758584,0.041765,0.073522,0.145238,0.221612,0.226593,0.064455,0.104308,0.105391,0.007884,0.027393,0.117567,0.087878,0.978092,0.754688,0.043901,0.077154,0.149343,0.225355,0.230208,0.066706,0.100734,0.107995,0.008351,0.028657,0.

１、DAYS_EMPLOYEDが異常値（365243）を取っている。それにより、平均も上がっている。  
　　　　→365243と０の割合を調べて、0がなければ365243が0であると推測し置き換える。  
２、objectは数値変換する  
３、DAYS_BIRTH、DAYS_REGISTRATION、DAYS_ID_PUBLISHの値がマイナスなので絶対値を取ってプラスに直す。

再度実行することにする。データは１０万件とする。

In [50]:
df_train_former = pd.read_csv('../application_train.csv')
df_train_former['DAYS_EMPLOYED'].where(df_train_former['DAYS_EMPLOYED']!=365243, 0, inplace=True)
df_train_former[['DAYS_BIRTH', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'DAYS_EMPLOYED']] = np.fabs(df_train_former[['DAYS_BIRTH', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'DAYS_EMPLOYED']])
df_train = df_train_former.sample(frac=1)[:100000]
df_X = df_train.drop("TARGET", axis=1)
df_Y = df_train[["TARGET"]]

#カテゴリーを数値変換
categorical_feats = [
    f for f in df_X.columns if df_X[f].dtype == 'object'
]
categorical_feats_ = categorical_feats.copy()

for f in categorical_feats_:
    df_X[f], _ = pd.factorize(df_X[f])
    df_X[f] = df_X[f].astype('int')
    
#残りの欠損値を平均で埋める
df_X = df_X.fillna(df_X.mean())

#データ分割
(X_train, X_test, 
     y_train, y_test) = train_test_split(df_X.values, df_Y.values.flatten(), test_size=0.3)

#標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [51]:
diparameter = {
    "C": [10**i for i in range(-2,4)],
    "random_state": [123],
    "class_weight": ['balanced', None],
}

grid_search = GridSearchCV(LogisticRegression(),
                           param_grid=diparameter,
                           cv=5,
                           scoring='roc_auc',
                          )
grid_search.fit(X_train, y_train)

/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3

/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.01, 0.1, 1, 10, 100, 1000], 'random_state': [123], 'class_weight': ['balanced', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [52]:
#最良の点数
grid_search.best_score_

0.7441432736796232

In [61]:
# 最良の方法で予測
y_predict_proba_logistic_gridsearched = grid_search.best_estimator_.predict_proba(X_test)

In [62]:
"""
テストデータを読み込み、処理
"""

df_apptest = pd.read_csv('../application_test.csv')

#カテゴリーを数値変換
categorical_feats = [
    f for f in df_apptest.columns if df_apptest[f].dtype == 'object'
]
categorical_feats_ = categorical_feats.copy()

for f in categorical_feats_:
    df_apptest[f], _ = pd.factorize(df_apptest[f])
    df_apptest[f] = df_apptest[f].astype('int')
    
#残りの欠損値を平均で埋める
df_apptest = df_apptest.fillna(df_apptest.mean())

X_apptest = df_apptest.values

#標準化
scaler.fit(X_apptest)
X_apptest = scaler.transform(X_apptest)

y_apt_predict_proba_logistic_gridsearched = grid_search.best_estimator_.predict_proba(X_apptest)

In [63]:
"""
csvファイルへ書き込み
"""

import csv as csv

submit_file = open("home_credit_submit_190203_1.csv", "w")
file_object = csv.writer(submit_file)
file_object.writerow(["SK_ID_CURR", "TARGET"])
file_object.writerows(zip(df_apptest["SK_ID_CURR"].values, y_apt_predict_proba_logistic_gridsearched[:, 1]))
submit_file.close()

In [56]:
#ランダムサーチで最良の方法を確認
param_dist = {
    "C": uniform(loc=0, scale=4),
    "penalty": ['l2'],
    "class_weight": ['balanced', None],
}

random_search = RandomizedSearchCV(
    LogisticRegression(),
   param_distributions=param_dist,
   cv=5,
   scoring='roc_auc',
)
random_search.fit(X_train, y_train)

/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3

/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a35e53358>, 'penalty': ['l2'], 'class_weight': ['balanced', None]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [57]:
y_predict_proba_logistic_random = random_search.best_estimator_.predict_proba(X_test)

In [59]:
"""
csvファイルへ書き込み
"""

submit_file = open("home_credit_submit_190203_2.csv", "w")
file_object = csv.writer(submit_file)
file_object.writerow(["SK_ID_CURR", "TARGET"])
file_object.writerows(zip(df_apptest["SK_ID_CURR"].values, y_apt_predict_proba_logistic_random[:, 1]))
submit_file.close()